In [1]:
#  APIs from Keras we will be using
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


In [2]:
"""
Here we declare all important parameters we will be using
throughout the development such as dimensions of images,
path to train & validation datasets etc...
"""
img_width, img_height = 150, 150
train_data_dir = '../dataset/train'
validation_data_dir = '../dataset/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [3]:
#  We need to check here how Keras.json is setup
#  Channels == 3 since we have RGB values in our pixels
#  channels_first means we img[0] indicates RGB images, img[1] is width of the image, img[2] is the height of the img
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
#  Instantiation of Keras Object for Convulation Neural Networks
model = Sequential()

#  the first layer needs to know what kind of shape to expect
#  next layers will automatically infer the dimensions
model.add(Conv2D(32, (3, 3), input_shape=input_shape))

#  here we specify function which filter will apply to the image 
model.add(Activation('relu'))

#  subsampling, we're specifying just the size of filter
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
#  this is so-called dense layer. It takes data produced by ConvNet
#  and applies yet another set of layers to come up with final one value.. 0 or 1
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [7]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [8]:
img = load_img('../dataset/train/cats/cat.199.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in train_datagen.flow(x, batch_size=1,
                          save_to_dir='../dataset/preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [ ]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')  # always save your weights after training or during training

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
125/125 [==============================] - 79s 629ms/step - loss: 0.7172 - acc: 0.5195 - val_loss: 0.6859 - val_acc: 0.5000
Epoch 2/50
125/125 [==============================] - 87s 697ms/step - loss: 0.6897 - acc: 0.5660 - val_loss: 0.6302 - val_acc: 0.6488
Epoch 3/50
125/125 [==============================] - 83s 664ms/step - loss: 0.6498 - acc: 0.6150 - val_loss: 0.6273 - val_acc: 0.6575
Epoch 4/50
125/125 [==============================] - 92s 736ms/step - loss: 0.6113 - acc: 0.6755 - val_loss: 0.6917 - val_acc: 0.6100
Epoch 5/50
125/125 [==============================] - 82s 656ms/step - loss: 0.6115 - acc: 0.6670 - val_loss: 0.5529 - val_acc: 0.7063
Epoch 6/50
125/125 [==============================] - 86s 690ms/step - loss: 0.5882 - acc: 0.7020 - val_loss: 0.6109 - val_acc: 0.6700
Epoch 7/50
125/125 [==============================] - 83s 6

In [ ]:
new_img = load_img("../dataset/validation/cats.199.jpg")
print(model.predict(new_img))